# Glass Sponge (Hexactinellid) Structural Motifs — Visualizer

This notebook is a small *interactive app* (built with **Gradio + Plotly**) that lets you visualize
parametric, schematic geometries inspired by classic structural biology / mechanics papers on **glass sea sponges** (Hexactinellida), especially *Euplectella aspergillum*:

- **Macro (whole skeleton):** cylindrical **square-grid lattice** with an **alternating (checkerboard) diagonal bracing** pattern, plus **external spiral ridges**, a **terminal sieve plate**, and a **basal anchor tuft**.
- **Micro/Meso (components):** laminated **spicule cross-sections** (alternating silica/organic layers around an axial filament), and **composite struts** (bundled spicules embedded in a silica matrix).
- **Element level:** a simplified **non‑planar cruciform spicule** element.

> These are **schematic** models meant for intuition and exploration (not anatomical reconstructions).

---

## Files created by this project

- `glass_sponge_algorithms.py` — the reusable geometry + plotting algorithms
- This notebook — a UI front-end around those algorithms

---

## Quick start

Run all cells, then use the tabs in the UI.


In [2]:
# If you are running this notebook in a clean environment, you may need:
# !pip install plotly gradio numpy

import sys
from pathlib import Path

# Ensure the folder containing this notebook is on sys.path
here = Path().resolve()
if str(here) not in sys.path:
    sys.path.insert(0, str(here))

import numpy as np
import plotly.graph_objects as go
import gradio as gr

from glass_sponge_algorithms import (
    # parameter containers
    LatticeParams, RidgeParams, SievePlateParams,
    SpiculeLamellaParams, CompositeBeamParams,
    AnchorBundleParams, CruciformSpiculeParams,
    NanoparticleRingParams,

    # figure builders
    lattice_figure, full_skeleton_figure,
    spicule_cross_section_figure, composite_beam_cross_section_figure,
    anchor_bundle_figure, cruciform_spicule_figure,
    nanoparticle_rings_figure,
)
print("Imports OK")


Imports OK


In [3]:
# -----------------------------
# Render helper functions
# -----------------------------

def render_macro_skeleton(
    radius: float,
    height: float,
    n_theta: int,
    n_z: int,
    diagonal_mode: str,
    diagonal_pair_offset: float,
    n_ridges: int,
    ridge_pitch: float,
    ridge_handedness: str,
    sieve_rings: int,
    sieve_spokes: int,
    anchor_fibers: int,
    anchor_length: float,
    anchor_spread: float,
    anchor_waviness: float,
):
    lattice = LatticeParams(
        radius=radius,
        height=height,
        n_theta=n_theta,
        n_z=n_z,
        diagonal_mode=diagonal_mode,
        diagonal_pair_offset=diagonal_pair_offset,
    )
    ridges = RidgeParams(
        n_ridges=n_ridges,
        pitch=ridge_pitch,
        handedness=ridge_handedness,
    )
    sieve = SievePlateParams(
        n_rings=sieve_rings,
        n_spokes=sieve_spokes,
        z_at_top=height,
    )
    anchor = AnchorBundleParams(
        n_fibers=anchor_fibers,
        length=anchor_length,
        spread=anchor_spread,
        waviness=anchor_waviness,
        seed=0,
    )
    return full_skeleton_figure(lattice, ridges, sieve, anchor, show_axes=False)


def render_lattice_only(radius: float, height: float, n_theta: int, n_z: int, diagonal_mode: str, diagonal_pair_offset: float):
    lattice = LatticeParams(
        radius=radius,
        height=height,
        n_theta=n_theta,
        n_z=n_z,
        diagonal_mode=diagonal_mode,
        diagonal_pair_offset=diagonal_pair_offset,
    )
    return lattice_figure(lattice, ridges=None, sieve=None, show_axes=False)


def render_spicule_cross_section(
    outer_radius: float,
    n_layers: int,
    organic_thickness: float,
    silica_th_in: float,
    silica_th_out: float,
    axial_filament_radius: float,
    thickness_profile: str,
    powerlaw_exp: float,
):
    p = SpiculeLamellaParams(
        outer_radius=outer_radius,
        n_silica_layers=n_layers,
        organic_thickness=organic_thickness,
        silica_thickness_inner=silica_th_in,
        silica_thickness_outer=silica_th_out,
        axial_filament_radius=axial_filament_radius,
        thickness_profile=thickness_profile,
        powerlaw_exp=powerlaw_exp,
    )
    return spicule_cross_section_figure(p, show_legend=False)


def render_composite_beam(
    beam_radius: float,
    n_spicules: int,
    r_mean: float,
    r_std: float,
    padding: float,
    seed: int,
):
    p = CompositeBeamParams(
        beam_radius=beam_radius,
        n_spicules=n_spicules,
        spicule_radius_mean=r_mean,
        spicule_radius_std=r_std,
        packing_padding=padding,
        seed=seed,
    )
    return composite_beam_cross_section_figure(p, show_legend=False)


def render_anchor_bundle(
    n_fibers: int,
    length: float,
    spread: float,
    waviness: float,
    seed: int,
):
    p = AnchorBundleParams(
        n_fibers=n_fibers,
        length=length,
        spread=spread,
        waviness=waviness,
        seed=seed,
    )
    return anchor_bundle_figure(p, show_axes=False)


def render_cruciform(ray_length: float, tilt_degrees: float, n_points_per_ray: int):
    p = CruciformSpiculeParams(
        ray_length=ray_length,
        tilt_degrees=tilt_degrees,
        n_points_per_ray=n_points_per_ray,
    )
    return cruciform_spicule_figure(p, show_axes=True)


def render_nanoparticle_rings(
    outer_radius: float,
    axial_filament_radius: float,
    n_rings: int,
    particles_per_ring: int,
    radial_jitter: float,
    angular_jitter: float,
    seed: int,
):
    p = NanoparticleRingParams(
        outer_radius=outer_radius,
        axial_filament_radius=axial_filament_radius,
        n_rings=n_rings,
        particles_per_ring=particles_per_ring,
        radial_jitter=radial_jitter,
        angular_jitter=angular_jitter,
        seed=seed,
    )
    return nanoparticle_rings_figure(p, show_legend=False)


In [ ]:
# -----------------------------
# Gradio UI
# -----------------------------

with gr.Blocks() as demo:
    gr.Markdown("## Glass Sponge Structural Visualizer (schematic)")

    with gr.Tab("Macro skeleton"):
        gr.Markdown("Cylindrical lattice + diagonal bracing + spiral ridges + sieve plate + anchor tuft.")
        with gr.Row():
            with gr.Column(scale=1):
                radius = gr.Slider(0.5, 3.0, value=1.0, step=0.1, label="Radius")
                height = gr.Slider(2.0, 10.0, value=5.0, step=0.1, label="Height")
                n_theta = gr.Slider(6, 48, value=24, step=1, label="Vertical struts (around)")
                n_z = gr.Slider(6, 60, value=30, step=1, label="Horizontal rings (along height)")
                diagonal_mode = gr.Dropdown(["none", "all_cells", "checkerboard"], value="checkerboard", label="Diagonal bracing pattern")
                diagonal_pair_offset = gr.Slider(0.0, 0.08, value=0.02, step=0.005, label="Paired diagonal offset (0 = single)")

                gr.Markdown("**Spiral ridges**")
                n_ridges = gr.Slider(1, 10, value=4, step=1, label="Number of ridges per handedness")
                ridge_pitch = gr.Slider(0.5, 6.0, value=2.0, step=0.1, label="Pitch (z per revolution)")
                ridge_handedness = gr.Dropdown(["both", "right", "left"], value="both", label="Handedness")

                gr.Markdown("**Sieve plate**")
                sieve_rings = gr.Slider(1, 12, value=6, step=1, label="Concentric rings")
                sieve_spokes = gr.Slider(6, 60, value=24, step=1, label="Radial spokes")

                gr.Markdown("**Anchor tuft**")
                anchor_fibers = gr.Slider(10, 200, value=80, step=1, label="Fibers")
                anchor_length = gr.Slider(0.5, 6.0, value=2.5, step=0.1, label="Length")
                anchor_spread = gr.Slider(0.1, 3.0, value=1.0, step=0.05, label="Spread")
                anchor_waviness = gr.Slider(0.0, 1.0, value=0.25, step=0.05, label="Waviness")

                btn = gr.Button("Render")
            with gr.Column(scale=2):
                out = gr.Plot(label="3D view")

        btn.click(
            render_macro_skeleton,
            inputs=[radius, height, n_theta, n_z, diagonal_mode, diagonal_pair_offset,
                    n_ridges, ridge_pitch, ridge_handedness,
                    sieve_rings, sieve_spokes,
                    anchor_fibers, anchor_length, anchor_spread, anchor_waviness],
            outputs=out,
        )

    with gr.Tab("Lattice only"):
        gr.Markdown("Square grid + diagonal bracing on a cylinder (macro framework).")
        with gr.Row():
            with gr.Column(scale=1):
                r2 = gr.Slider(0.5, 3.0, value=1.0, step=0.1, label="Radius")
                h2 = gr.Slider(2.0, 10.0, value=5.0, step=0.1, label="Height")
                nt2 = gr.Slider(6, 48, value=24, step=1, label="Vertical struts (around)")
                nz2 = gr.Slider(6, 60, value=30, step=1, label="Horizontal rings (along height)")
                dm2 = gr.Dropdown(["none", "all_cells", "checkerboard"], value="checkerboard", label="Diagonal mode")
                dpo2 = gr.Slider(0.0, 0.08, value=0.02, step=0.005, label="Paired diagonal offset")
                btn2 = gr.Button("Render")
            with gr.Column(scale=2):
                out2 = gr.Plot(label="3D view")
        btn2.click(render_lattice_only, inputs=[r2, h2, nt2, nz2, dm2, dpo2], outputs=out2)

    with gr.Tab("Spicule lamellae"):
        gr.Markdown("Concentric silica layers separated by organic interlayers around a central axial filament (cross-section schematic).")
        with gr.Row():
            with gr.Column(scale=1):
                sr = gr.Slider(0.3, 2.0, value=1.0, step=0.05, label="Outer radius")
                nl = gr.Slider(3, 40, value=18, step=1, label="Number of silica layers")
                org = gr.Slider(0.0, 0.08, value=0.02, step=0.005, label="Organic interlayer thickness")
                th_in = gr.Slider(0.01, 0.3, value=0.12, step=0.01, label="Silica thickness (inner)")
                th_out = gr.Slider(0.005, 0.2, value=0.03, step=0.005, label="Silica thickness (outer)")
                afr = gr.Slider(0.0, 0.2, value=0.05, step=0.005, label="Axial filament radius")
                prof = gr.Dropdown(["linear", "powerlaw"], value="linear", label="Thickness profile")
                pexp = gr.Slider(0.5, 3.0, value=1.4, step=0.1, label="Powerlaw exponent (if used)")
                btn3 = gr.Button("Render")
            with gr.Column(scale=2):
                out3 = gr.Plot(label="2D cross-section")
        btn3.click(render_spicule_cross_section, inputs=[sr, nl, org, th_in, th_out, afr, prof, pexp], outputs=out3)

    with gr.Tab("Nano: silica nanospheres"):
        gr.Markdown("Schematic view of silica nanospheres arranged in concentric rings (nano-scale motif).")
        with gr.Row():
            with gr.Column(scale=1):
                nor = gr.Slider(0.3, 2.0, value=1.0, step=0.05, label="Outer radius")
                ncore = gr.Slider(0.0, 0.3, value=0.08, step=0.01, label="Axial filament radius")
                nring = gr.Slider(1, 30, value=10, step=1, label="Number of rings")
                ppr = gr.Slider(6, 120, value=40, step=1, label="Particles per ring")
                rjit = gr.Slider(0.0, 0.15, value=0.02, step=0.005, label="Radial jitter")
                ajit = gr.Slider(0.0, 0.25, value=0.04, step=0.005, label="Angular jitter")
                nseed = gr.Slider(0, 50, value=0, step=1, label="Seed")
                btnN = gr.Button("Render")
            with gr.Column(scale=2):
                outN = gr.Plot(label="2D view")
        btnN.click(render_nanoparticle_rings, inputs=[nor, ncore, nring, ppr, rjit, ajit, nseed], outputs=outN)

    with gr.Tab("Composite strut"):
        gr.Markdown("Bundle of spicules embedded in a matrix (cross-section schematic).")
        with gr.Row():
            with gr.Column(scale=1):
                br = gr.Slider(0.3, 2.0, value=1.0, step=0.05, label="Beam radius")
                ns = gr.Slider(1, 80, value=20, step=1, label="Spicules in bundle")
                rm = gr.Slider(0.02, 0.4, value=0.12, step=0.01, label="Mean spicule radius")
                rs = gr.Slider(0.0, 0.2, value=0.03, step=0.005, label="Spicule radius std")
                pad = gr.Slider(0.0, 0.05, value=0.01, step=0.005, label="Padding")
                seed = gr.Slider(0, 50, value=0, step=1, label="Random seed")
                btn4 = gr.Button("Render")
            with gr.Column(scale=2):
                out4 = gr.Plot(label="2D cross-section")
        btn4.click(render_composite_beam, inputs=[br, ns, rm, rs, pad, seed], outputs=out4)

    with gr.Tab("Anchor tuft"):
        gr.Markdown("Flexible cluster of anchor spicules (schematic).")
        with gr.Row():
            with gr.Column(scale=1):
                nf = gr.Slider(10, 250, value=80, step=1, label="Fibers")
                al = gr.Slider(0.5, 8.0, value=2.5, step=0.1, label="Length")
                asp = gr.Slider(0.1, 4.0, value=1.0, step=0.05, label="Spread")
                aw = gr.Slider(0.0, 1.0, value=0.25, step=0.05, label="Waviness")
                sd = gr.Slider(0, 50, value=0, step=1, label="Seed")
                btn5 = gr.Button("Render")
            with gr.Column(scale=2):
                out5 = gr.Plot(label="3D view")
        btn5.click(render_anchor_bundle, inputs=[nf, al, asp, aw, sd], outputs=out5)

    with gr.Tab("Cruciform spicule element"):
        gr.Markdown("Non-planar cruciform spicule element (schematic building block).")
        with gr.Row():
            with gr.Column(scale=1):
                rl = gr.Slider(0.2, 3.0, value=1.0, step=0.1, label="Ray length")
                tilt = gr.Slider(0.0, 45.0, value=18.0, step=1.0, label="Non-planarity (tilt degrees)")
                npr = gr.Slider(5, 80, value=30, step=1, label="Points per ray")
                btn6 = gr.Button("Render")
            with gr.Column(scale=2):
                out6 = gr.Plot(label="3D view")
        btn6.click(render_cruciform, inputs=[rl, tilt, npr], outputs=out6)

    gr.Markdown(
        "### Notes\n"
        "- The geometry is **parametric and schematic**. The goal is to help you explore architectural ideas.\n"
        "- For publication-quality renderings or finite-element meshes, adapt `glass_sponge_algorithms.py` to export surfaces/solids."
    )

demo.launch(inline=True, debug=True, share=False, prevent_thread_lock=True)


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.
